# 「分散データ並列と分散RPCフレームワークの連携」

【原題】Combining Distributed DataParallel with Distributed RPC Framework

【原著】[Pritam Damania](https://github.com/pritamdamania87)

【元URL】https://pytorch.org/tutorials/advanced/rpc_ddp_tutorial.html

【翻訳】電通国際情報サービスISID HCM事業部　櫻井 亮佑

【日付】2020年12月10日

【チュトーリアル概要】

本チュートリアルではシンプルな例に対して、 [DistributedDataParallel](https://pytorch.org/docs/stable/nn.html#torch.nn.parallel.DistributedDataParallel)（DDP）と[分散RPCフレームワーク](https://pytorch.org/docs/master/rpc.html)の両方を利用し、分散データ並列化と分散モデル並列化を連携させて訓練する方法を解説します。<br>
なお、本チュートリアルで使用しているコードは、 [こちら](https://github.com/pytorch/examples/tree/master/distributed/rpc/ddp_rpc)から確認できます。

--- 

ここまでのチュートリアルである [分散データ並列訓練入門](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html)（日本語版6_3） と [分散RPCフレームワーク入門](https://pytorch.org/tutorials/intermediate/rpc_tutorial.html) （日本語版6_5）では、分散データ並列と分散モデル並列訓練をそれぞれ個別に実施する方法について解説しました。




しかし、これら2つのテクニックを組み合わせたくなるような訓練パラダイムも存在します。例えば、以下のようなケースです。

1. 疎な部分（大規模な埋め込みテーブル）と密な部分（FC層）を伴うモデルを扱う場合、埋め込みテーブルをパラメーターサーバー上に配置し、[DistributedDataParallel](https://pytorch.org/docs/stable/nn.html#torch.nn.parallel.DistributedDataParallel) を使ってFC層を複数のトレーナーに渡って複製したくなるかもしれません。この場合、パラメーターサーバー上で埋め込みの参照を行う際に、[分散RPCフレームワーク](https://pytorch.org/docs/master/rpc.html) が使用できます。


2. [PipeDream](https://arxiv.org/abs/1806.03377) の論文にあるハイブリッド並列化を実現します。[分散RPCフレームワーク](https://pytorch.org/docs/master/rpc.html) を使用してモデルのステージを複数ワーカーに渡ってパイプライン化し、（必要であれば）[DistributedDataParallel](https://pytorch.org/docs/stable/nn.html#torch.nn.parallel.DistributedDataParallel) によって各ステージを複製することが可能です。

本チュートリアルでは、上記に掲載したケースのうち、1番目を扱います。

今回の構成では、以下に示す計4つのワーカーを使用します。

1. マスター。パラメーターサーバー上で埋め込みテーブル（nn.EmbeddingBag）を作成する役目を持ちます。また、マスターは2つのトレーナー上で訓練ループを主導します。

2. パラメーターサーバー。基本的にはメモリ内で埋め込みテーブルを保持し、マスターとトレーナーからのRPCに応答します。

3. 2つのトレーナー。 [DistributedDataParallel](https://pytorch.org/docs/stable/nn.html#torch.nn.parallel.DistributedDataParallel) を使い、2つのトレーナー間で複製されるFC層（nn.Linear）を格納します。またトレーナーは、フォワードパス、バックワードパス、そしてオプティマイザーステップを実行する役割も担います。

そして、訓練プロセス全体は以下のように実行されます。

1. マスターがパラメーターサーバー上に埋め込みテーブルを作成し、この埋め込みテーブルへの [RRef](https://pytorch.org/docs/master/rpc.html#rref) を保持します。

2. マスターがトレーナーに訓練ループを開始させ、埋め込みテーブルのRRefをトレーナーに渡します。

3. 2つのトレーナーが、マスターから渡された埋め込みテーブルのRRefを使用して、埋め込みの参照を行う `HybridModel` を初めに作成し、その後DDPの内部にラップされているFC層を実行します。

4. トレーナーがモデルのフォワードパスを実行し、[分散自動微分](https://pytorch.org/docs/master/rpc.html#distributed-autograd-framework) を使って、損失をもとに誤差逆伝搬を実行します。

5. 誤差逆伝搬の内部処理として、FC層の勾配が初めに計算され、DDP内のallreduceを通してすべての訓練トレーナーに同期されます。

6. 分散自動微分が勾配情報をパラメーターサーバーに渡し、パラメーターサーバー上で埋め込みテーブルの勾配が更新されます。

7. 最後に、[分散オプティマイザー](https://pytorch.org/docs/master/rpc.html#module-torch.distributed.optim) を使用して、すべてのパラメーターを更新します。

**・注意**<br>
DDPとRPCを組み合わせる場合、バックワードパス（誤差逆伝搬）では常に、[分散自動微分](https://pytorch.org/docs/master/rpc.html#distributed-autograd-framework) を使用するべきです。

では、各パートの詳細を解説します。

訓練を行う前に、今回使用するすべてのワーカーを初めに準備する必要があります。

4つのプロセスを作成し、ランク0とランク1をトレーナー、ランク2をマスター、そしてランク3をパラメーターサーバーとします。

TCPの init_method を使用して、4つのワーカー上すべてでRPCフレームワークを初期化します。


RPCの初期化完了後、マスターは [rpc.remote](https://pytorch.org/docs/master/rpc.html#torch.distributed.rpc.remote) を使用してパラメーターサーバー上に [EmbeddingBag](https://pytorch.org/docs/master/generated/torch.nn.EmbeddingBag.html) を作成します。


そしてマスターは、各トレーナーをループし、[rpc_async](https://pytorch.org/docs/master/rpc.html#torch.distributed.rpc.rpc_async) により各トレーナー上で `_run_trainer` を呼び出し、訓練ループを開始します。

最後に、マスターは離脱前にすべての訓練が終了するまで待機します。

トレーナーは初めに、[init_process_group](https://pytorch.org/docs/stable/distributed.html#torch.distributed.init_process_group) を使用し、world_size=2（2つのトレーナー用）でDDPのための `ProcessGroup` を初期化します。

次に、トレーナーはTCPの `init_method` を使用してRPCフレームワークを初期化します。

なお、RPCの初期化と`ProcessGroup`の初期化処理でぽポートが異なる点に注意してください。

これは、両フレームワークの初期化処理の間でポートの競合が発生することを防ぐためです。

初期化が完了した後は、トレーナーはただマスターから来る `_run_trainer` のRPCを待機するだけです。

パラメーターサーバーは、RPCフレームワークを初期化し、トレーナーとマスターからのRPCを待機するのみです。

In [ ]:
def run_worker(rank, world_size):
    r"""
    RPCを初期化するラッパー関数。
    関数を呼び出し、RPCを停止します。
    """
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '29500'


    rpc_backend_options = TensorPipeRpcBackendOptions()
    rpc_backend_options.init_method='tcp://localhost:29501'

    # ランク2はmaster、ランク3はps、ランク0とランク1はトレーナーです。
    if rank == 2:
        rpc.init_rpc(
                "master",
                rank=rank,
                world_size=world_size,
                rpc_backend_options=rpc_backend_options)

        # ps上に埋め込みテーブルを構築します。
        emb_rref = rpc.remote(
                "ps",
                torch.nn.EmbeddingBag,
                args=(NUM_EMBEDDINGS, EMBEDDING_DIM),
                kwargs={"mode": "sum"})

        # 2つのトレーナー上で訓練ループを実行します。
        futs = []
        for trainer_rank in [0, 1]:
            trainer_name = "trainer{}".format(trainer_rank)
            fut = rpc.rpc_async(
                    trainer_name, _run_trainer, args=(emb_rref, rank))
            futs.append(fut)

        # すべての訓練が終了するまで待機します。
        for fut in futs:
            fut.wait()
    elif rank <= 1:
        # 2つのトレーナー上で分散データ並列を行うため、プロセスグループを初期化します。
        dist.init_process_group(
                backend="gloo", rank=rank, world_size=2)

        # RPCを初期化します。
        trainer_name = "trainer{}".format(rank)
        rpc.init_rpc(
                trainer_name,
                rank=rank,
                world_size=world_size,
                rpc_backend_options=rpc_backend_options)

        # トレーナーはマスターからのRPCを待機します。
    else:
        rpc.init_rpc(
                "ps",
                rank=rank,
                world_size=world_size,
                rpc_backend_options=rpc_backend_options)
        # パラメーターサーバーは特に何も処理を行いません。
        pass

    # すべてのrpcが終了するまで、ブロックします。
    rpc.shutdown()


if __name__=="__main__":
    # 2つのトレーナー、1つのパラメーターサーバー、1つのマスターです。
    world_size = 4
    mp.spawn(run_worker, args=(world_size, ), nprocs=world_size, join=True)

Trainerの詳細について解説する前に、トレーナーが使用する `HybridModel` の紹介しましす。

下記に示すように、`HybridModel` は、パラメーターサーバー上の埋め込みテーブル（`emb_rref`）のRRefとDDPに使用する `device` を用いて初期化されます。

モデルの初期化処理では、複製するためにDDP内の [nn.Linear](https://pytorch.org/docs/master/generated/torch.nn.Linear.html) 層をラップし、ラップしたこの線形層をすべてのトレーナー間で同期するようにします。


モデルのフォワードメソッドはとてもシンプルです。

[RRefのヘルパー関数](https://pytorch.org/docs/master/rpc.html#torch.distributed.rpc.RRef.rpc_sync) を使用してパラメーターサーバー上の埋め込みの参照を行い、得られた出力をFC層に渡します。

In [ ]:
class HybridModel(torch.nn.Module):
    """
    モデルは、疎な部分と密な部分で構成されています。
    密な部分は、分散データ並列を用いてすべてのトレーナー上に複製される nn.Linear モジュールです。
    疎な部分は、パラメーターサーバー上に格納されている nn.EmbeddingBag です。
    
    モデルは、パラメーターサーバー上の埋め込みテーブルへのリモート参照を保持しています。
    """

    def __init__(self, emb_rref, device):
        super(HybridModel, self).__init__()
        self.emb_rref = emb_rref
        self.fc = DDP(torch.nn.Linear(16, 8).cuda(device), device_ids=[device])
        self.device = device

    def forward(self, indices, offsets):
        emb_lookup = self.emb_rref.rpc_sync().forward(indices, offsets)
        return self.fc(emb_lookup.cuda(self.device))

次に、Trainerでのセットアップを確認しましょう。


トレーナーは、パラメーターサーバー上の埋め込みテーブルへのRRefと自身のランクを使用して、上掲の `HybridModel` を初めに作成します。

そして、[DistributedOptimizer](https://pytorch.org/docs/master/rpc.html#module-torch.distributed.optim) で最適化したいすべてのパラメーターへのRRefのリストを回収する必要があります。

パラメーターサーバーから埋め込みテーブルのパラメーターを回収するために、`_retrieve_embedding_parameters` という単純な関数を定義します。

この関数は、基本的には埋め込みテーブルのすべてのパラメーターを走査し、RRefのリストを返します。

トレーナーはパラメーターサーバー上のこのメソッドをRPC経由で呼び出し、必要なパラメーターへのRRefのリストを回収します。



また、`DistributedOptimizer` は、最適化される必要のあるパラメーターへのRRefのリストを常に引数に取るため、FC層のために、ローカルパラメーターであってもRRefを作成しなければいけません。

これは、`model.parameters()` を走査して各パラメーターへのRRefを作成し、リストに追加することで行なえます。

なお、`model.parameters()` はローカルパラメーターのみを返し、`emb_rref` は返り値に含まれていない点に注意してください。

最後に、すべてのRRefを使用して `DistributedOptimizer` を作成し、`CrossEntropyLoss` 関数を定義します。

In [ ]:
def _retrieve_embedding_parameters(emb_rref):
    param_rrefs = []
    for param in emb_rref.local_value().parameters():
        param_rrefs.append(RRef(param))
    return param_rrefs


def _run_trainer(emb_rref, rank):
    r"""
    各トレーナーは、パラメーターサーバー上の埋め込みの参照と
    ローカルでの nn.Linear の実行を含むフォワードパスを実行します。
    バックワードパスの間は、DDPが密な部分（nn.Linear）の勾配の集約の役割を担い、
    分散自動微分が、勾配の更新がパラメーターサーバーに伝播されるように担保しています。
    """

    # モデルをセットアップします。
    model = HybridModel(emb_rref, rank)

    # DistributedOptimizer のために rref である モデルのすべてのパラメーターを回収します。

    # 埋め込みテーブル用にパラメーターを回収します。
    model_parameter_rrefs = rpc.rpc_sync(
            "ps", _retrieve_embedding_parameters, args=(emb_rref,))

    # model.parameters() は、ローカルパラメーターのみを含んでいます。
    for param in model.parameters():
        model_parameter_rrefs.append(RRef(param))

    # 分散オプティマイザーをセットアップします。
    opt = DistributedOptimizer(
        optim.SGD,
        model_parameter_rrefs,
        lr=0.05,
    )

    criterion = torch.nn.CrossEntropyLoss()

各トレーナー上で実行する訓練ループを導入する準備ができました。

`get_next_batch` は、訓練用にランダムな入力とターゲットを生成する補助関数（ヘルパー関数）です。

そして、訓練ループを複数エポック実行します。



各バッチでは以下の処理を行います。

1. 分散自動微分のために [分散自動微分コンテクスト](https://pytorch.org/docs/master/rpc.html#torch.distributed.autograd.context) をセットアップします。

2. モデルのフォワードパスを実行し、出力を取得します。

3. 取得した出力とターゲットを基に、損失関数を用いて損失を計算します。

4. 損失を使用して、分散自動微分により分散バックワードパスを実行します。

5. 最後に、分散オプティマイザーでステップを実行し、すべてのパラメーターを最適化します。

In [ ]:
    def get_next_batch(rank):
        for _ in range(10):
            num_indices = random.randint(20, 50)
            indices = torch.LongTensor(num_indices).random_(0, NUM_EMBEDDINGS)

            # オフセットの生成
            offsets = []
            start = 0
            batch_size = 0
            while start < num_indices:
                offsets.append(start)
                start += random.randint(1, 10)
                batch_size += 1

            offsets_tensor = torch.LongTensor(offsets)
            target = torch.LongTensor(batch_size).random_(8).cuda(rank)
            yield indices, offsets_tensor, target

    # 100エポックの訓練
    for epoch in range(100):
        # 分散自動微分コンテクストの作成
        for indices, offsets, target in get_next_batch(rank):
            with dist_autograd.context() as context_id:
                output = model(indices, offsets)
                loss = criterion(output, target)

                # 分散バックワードパスの実行
                dist_autograd.backward(context_id, [loss])

                # 分散オプティマイザーの実行
                opt.step(context_id)

                # 各イテレーションで、別の勾配を管理する異なる分散自動微分コンテクストを作成するため、
                # 勾配をゼロ化する必要はありません。
        print("Training done for epoch {}".format(epoch))

以上となります。サンプルのソースコードの全体は、[こちら](https://github.com/pytorch/examples/tree/master/distributed/rpc/ddp_rpc)から確認できます。